<a href="https://colab.research.google.com/github/Nacho2904/orga_de_datos/blob/main/tp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico II: Spark

## Setup

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 52 kB/s 
     |████████████████████████████████| 199 kB 68.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=05e8a931e87eb107053979939fcffaab8281e57d32bddbe48d6b26d78f718e2d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive 

drive.mount('/content/gdrive')
path_a_cast = 'gdrive/MyDrive/Parquets/cast.parquet'
path_a_companies = 'gdrive/MyDrive/Parquets/companies.parquet'
path_a_crew = 'gdrive/MyDrive/Parquets/crew.parquet'
path_a_actores = 'gdrive/MyDrive/Parquets/imdb_actors.parquet'
path_a_keywords = 'gdrive/MyDrive/Parquets/keywords.parquet'
path_a_links = 'gdrive/MyDrive/Parquets/links.parquet'
path_a_movies = 'gdrive/MyDrive/Parquets/movies.parquet'
path_a_ratings = 'gdrive/MyDrive/Parquets/ratings.parquet'


Mounted at /content/gdrive


In [18]:
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


## S7

¿Cuál fue el año más mortal para los actores y actrices?

In [123]:
AÑO_FALLECIMIENTO = 3
CONTEO_MUERTES = 1
rdd_actores = sqlContext.read.parquet(path_a_actores).rdd
rdd_actores.take(5)

[Row(nconst='nm0000001', primaryName='Fred Astaire', birthYear='1899', deathYear='1987', primaryProfession='soundtrack,actor,miscellaneous', knownForTitles='tt0053137,tt0050419,tt0031983,tt0072308'),
 Row(nconst='nm0000002', primaryName='Lauren Bacall', birthYear='1924', deathYear='2014', primaryProfession='actress,soundtrack', knownForTitles='tt0117057,tt0037382,tt0071877,tt0038355'),
 Row(nconst='nm0000003', primaryName='Brigitte Bardot', birthYear='1934', deathYear='\\N', primaryProfession='actress,soundtrack,music_department', knownForTitles='tt0054452,tt0049189,tt0056404,tt0057345'),
 Row(nconst='nm0000004', primaryName='John Belushi', birthYear='1949', deathYear='1982', primaryProfession='actor,soundtrack,writer', knownForTitles='tt0072562,tt0080455,tt0077975,tt0078723'),
 Row(nconst='nm0000005', primaryName='Ingmar Bergman', birthYear='1918', deathYear='2007', primaryProfession='writer,director,actor', knownForTitles='tt0083922,tt0050976,tt0050986,tt0060827')]

In [124]:
rdd_años_de_fallecimiento = rdd_actores.map(lambda row: row[AÑO_FALLECIMIENTO])
rdd_años_de_fallecimiento.first()

'1987'

In [125]:
rdd_años_de_fallecimiento = rdd_años_de_fallecimiento.filter(str.isnumeric)
rdd_años_de_fallecimiento = rdd_años_de_fallecimiento.map(int)
dict_con_conteo = dict(rdd_años_de_fallecimiento.countByValue())
año_mas_mortal = sorted([[llave, dict_con_conteo[llave]] for llave in dict_con_conteo], 
                        key = lambda año_y_conteo: año_y_conteo[CONTEO_MUERTES])[-1]
print(año_mas_mortal)

[2021, 3409]


## S23

In [ ]:
rdd_actores = sqlContext.read.parquet(path_a_actores).rdd
rdd_actores.take(5)

## S25

## S11

 Según los cinéfilos, ¿cuál es el rating promedio de las 5 películas más populares? Un usuario es cinéfilo cuando puntuó más de 50 películas y todas las puntuaciones son de más de 2.5 estrellas. Indicar id, título, popularidad y rating promedio de la película.

In [112]:
rdd_ratings = sqlContext.read.parquet(path_a_ratings).rdd
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd

In [113]:
rdd_ratings_key_value = rdd_ratings.map(lambda row: (row[0], [row[1], row[2]]))

In [114]:
get_mean_rating = lambda user_iterator: np.mean(np.array(list(user_iterator))[:,1])
user_mean_rating_and_total_ratings = lambda user, user_iterator: (user, [get_mean_rating(user_iterator), len(user_iterator)])
rdd_ratings_grouped_per_user = rdd_ratings_key_value.groupByKey().map(lambda row: user_mean_rating_and_total_ratings(row[0], row[1]))

In [159]:
rdd_cinefilos = rdd_ratings_grouped_per_user.filter(lambda user: user[1] > 2.5 and user[2]).map(lambda user: (user[0], ''))
rdd_ratings_key_value_cinefilos = rdd_ratings_key_value.join(rdd_cinefilos).map(lambda x: (x[0],x[1][0]))

In [140]:
rdd_ratings_key_value_cinefilos.take(1)

[(8020, [168, 4.0])]

In [153]:
popularity = 10
rdd_top_5_most_popular_movies = rdd_movies.filter(lambda row: row[10])
rdd_top_5_most_popular_movies = rdd_top_5_most_popular_movies.takeOrdered(5, key = lambda row: -row[popularity])

In [158]:
rdd_top_5_most_popular_movies[0][10]

547.488298

## S34